In [1]:
import os
import sys

module_path = os.path.abspath("..")
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import set_random_seed
set_random_seed()
    
import tensorflow as tf
from tensorflow.keras import backend as K

import numpy as np

# MultipleNgramConvs like in PACRR

https://arxiv.org/abs/1704.03940

In [2]:
from layers.interaction import SemanticInteractions
from layers.local_relevance import MultipleNgramConvs

In [3]:
# test
K.clear_session()

emb_matrix = np.random.random((10000,68))

input_query = tf.keras.layers.Input((8,), dtype="int32")
input_sentence = tf.keras.layers.Input((4,), dtype="int32")

semantic_interaction = SemanticInteractions(emb_matrix)
extendedpacrr = MultipleNgramConvs(max_ngram=3, k_max=2)

_out = semantic_interaction([input_query, input_sentence])
_out = extendedpacrr(_out)

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()

[EMBEDDING MATRIX SHAPE] (10000, 68)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
semantic_interactions (Semantic (None, 8, 4, 3)      136         input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
multiple_ngram_convs (MultipleN (None, 8, 18)        2352

In [4]:
# test with some values

query = np.random.randint(1,4,(1, 8))
document = np.random.randint(1,4,(1, 4))

y = model.predict([query, document])
print("print colums from the matrix")
print(y[0,:,0])
print(y[0,:,1])
print(y[0,:,2])
print(y[0,:,-2])
print(y[0,:,-1])

print colums from the matrix
[ 7.5661864 10.738802  10.738802   9.411268  10.738802   7.5661864
  9.411268   7.5661864]
[ 7.5661864 10.738802  10.738802   7.570742  10.738802   7.5661864
  7.570742   7.5661864]
[-1.363623  -1.5093865 -1.5093865 -1.5664757 -1.5093865 -1.363623
 -1.5664757 -1.363623 ]
[5.3002057 7.367325  7.5272017 7.540059  6.7676516 6.925189  6.0865474
 4.6671877]
[5.126718  7.314978  7.4037476 7.2781425 6.7188783 6.7726593 6.036393
 4.511224 ]


In [5]:
query[:,4:] = np.zeros(4,)
document[:,2:] = np.zeros(2,)

y = model.predict([query, document])
print("print colums from the matrix")
print(y[0,:,0])
print(y[0,:,1])
print(y[0,:,2])
print(y[0,:,-2])
print(y[0,:,-1])

print colums from the matrix
[ 7.5661864 10.738802  10.738802   7.570742   0.         0.
  0.         0.       ]
[ 7.5661864 10.738802  10.738802   7.570742   0.         0.
  0.         0.       ]
[0. 0. 0. 0. 0. 0. 0. 0.]
[5.3002057 7.314978  6.9820356 5.0388074 0.        0.        0.
 0.       ]
[4.3984413 6.0978746 6.5127707 4.0440497 0.        0.        0.
 0.       ]


# MaskedSoftmax

In [6]:
from layers.local_relevance import MaskedSoftmax

In [7]:
K.clear_session()


query = np.random.randint(1,8,(1, 10)).astype(np.float32)
query[:,5:] = np.zeros(5,)
idf_softmax = MaskedSoftmax()

print(query)
print(idf_softmax(query))

norm = query/np.sum(query)
print("norm",norm)
print(idf_softmax(norm))


[[5. 3. 1. 1. 5. 0. 0. 0. 0. 0.]]
tf.Tensor(
[[0.46041223 0.06231002 0.00843274 0.00843274 0.46041223 0.
  0.         0.         0.         0.        ]], shape=(1, 10), dtype=float32)
norm [[0.33333334 0.2        0.06666667 0.06666667 0.33333334 0.
  0.         0.         0.         0.        ]]
tf.Tensor(
[[0.22691017 0.19858573 0.17379694 0.17379694 0.22691017 0.
  0.         0.         0.         0.        ]], shape=(1, 10), dtype=float32)


In [8]:
## model simple outputs

K.clear_session()

input_query = tf.keras.layers.Input((10,), dtype="float32")

idf_softmax = MaskedSoftmax()

_out = idf_softmax(input_query)

model = tf.keras.models.Model(inputs=input_query, outputs=_out)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
masked_softmax (MaskedSoftma (None, 10)                0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [9]:

print(query)
print(model.predict(query))

norm = query/np.sum(query)
print("norm",norm)
print(model.predict(norm))

[[5. 3. 1. 1. 5. 0. 0. 0. 0. 0.]]
[[0.46041223 0.06231002 0.00843274 0.00843274 0.46041223 0.
  0.         0.         0.         0.        ]]
norm [[0.33333334 0.2        0.06666667 0.06666667 0.33333334 0.
  0.         0.         0.         0.        ]]
[[0.22691017 0.19858573 0.17379694 0.17379694 0.22691017 0.
  0.         0.         0.         0.        ]]
